In [ ]:
!pip install pyspark
!pip install pyngrok

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyngrok import ngrok

spark = SparkSession.builder.appName("PageRank").getOrCreate()
def parse_line(line):
    parts = line.split(":")
    node = int(parts[0].strip())
    links = eval(parts[1].strip())
    return (node, links)

lines = spark.sparkContext.textFile("question3.txt")
graph = lines.map(parse_line).collectAsMap()
num_nodes = len(graph)
initial_rank = 1.0 / num_nodes
ranks = {node: initial_rank for node in graph}
damping_factor = 0.85
num_iterations = 10
for _ in range(num_iterations):
    contributions = (
        spark.sparkContext.parallelize(graph.items())
        .flatMap(lambda x: [(neighbor, ranks[x[0]] / len(x[1])) for neighbor in x[1]])
        .reduceByKey(lambda a, b: a + b)
    )
    ranks = contributions.collectAsMap()
    for node in graph.keys():
        ranks[node] = (1 - damping_factor) / num_nodes + damping_factor * ranks.get(node, 0)
ranks_df = spark.createDataFrame([Row(node=node, rank=rank) for node, rank in ranks.items()])
highest_rank_node = ranks_df.orderBy("rank", ascending=False).first()
lowest_rank_node = ranks_df.orderBy("rank").first()
print(f"Highest PageRank: {highest_rank_node}")
print(f"Lowest PageRank: {lowest_rank_node}")

ngrok.set_auth_token("2oY32y1o2wSGEJe5t6a6WZAnWxW_7cMg48xpKAosLShJ5LDdX")
public_url = ngrok.connect(4040)
print(public_url)


Highest PageRank: Row(node=50, rank=0.026800733975577556)
Lowest PageRank: Row(node=44, rank=0.0015000000000000002)
NgrokTunnel: "https://8bc0-35-229-224-229.ngrok-free.app" -> "http://localhost:4040"
